<a href="https://colab.research.google.com/github/VishalF5/anime-scraping-/blob/main/anime_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import numpy as np
ur=[]
# #we are collecting url for all annime present on site
# #range is from 1 to 530 because 530 pages are there
# for i in range(1,530):
#     url='https://www.anime-planet.com/anime/all?page={}'.format(i)
#     headers = {
#             'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
#             }
#     response=requests.get(url,headers=headers)
#     con=response.content
#     soup=BeautifulSoup(con ,'lxml')
#     for name in soup.find_all('li',class_="card"):
#         for na in name.find_all('a'):
#             ur.append(na.get('href'))

In [ ]:
df1=pd.read_csv('animeplanet_url.csv')
# df=pd.DataFrame()


In [ ]:
df1['url']

0                                      /anime/spy-x-family
1        /anime/demon-slayer-kimetsu-no-yaiba-entertain...
2                    /anime/fruits-basket-the-final-season
3                   /anime/fullmetal-alchemist-brotherhood
4                                   /anime/mo-dao-zu-shi-3
                               ...                        
18510                     /anime/mo-jing-lieren-3rd-season
18511                       /anime/lao-fu-zi-shui-hu-chuan
18512                      /anime/shoot-goal-to-the-future
18513    /anime/bishoujo-senshi-sailor-moon-cosmos-movie-2
18514                                /anime/suisoh-suicide
Name: url, Length: 18515, dtype: object

In [ ]:
# df.to_csv("animeplanet_url.csv")

In [ ]:
col_names=['url',"Name",'Japanese_name','Type','Episodes','Studio','Release_season','Tags','Rating','Rank','Release_year','End_year','Description','Content_Warning','Related_Mange','Related_anime','Voice_actors','staff','Img']
df=pd.DataFrame(columns=col_names)

In [ ]:
df['url']=df1['url']

In [ ]:
df['url']

0                                      /anime/spy-x-family
1        /anime/demon-slayer-kimetsu-no-yaiba-entertain...
2                    /anime/fruits-basket-the-final-season
3                   /anime/fullmetal-alchemist-brotherhood
4                                   /anime/mo-dao-zu-shi-3
                               ...                        
18510                     /anime/mo-jing-lieren-3rd-season
18511                       /anime/lao-fu-zi-shui-hu-chuan
18512                      /anime/shoot-goal-to-the-future
18513    /anime/bishoujo-senshi-sailor-moon-cosmos-movie-2
18514                                /anime/suisoh-suicide
Name: url, Length: 18515, dtype: object

In [ ]:
i=0
for u in df['url']:
    url="https://www.anime-planet.com{}".format(u)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
        }
    response=requests.get(url,headers=headers)
    con=response.content
    soup=BeautifulSoup(con ,'lxml')

    #print(i)
    print(url)

    #names
    eng_name=[]
    if soup.find_all(itemprop='name'):
        for en in soup.find_all(itemprop='name'):
            if en!=None:
                eng_name.append(en.text.strip())
        df.iloc[i,1]=eng_name[0]
    else:
        df.iloc[i,1]=np.nan
    

    jap_name=[]
    if soup.find(class_='aka'):
        for name in soup.find(class_='aka'):
            jap_name.append(name.strip())
        df.iloc[i,2]=jap_name[0][11:]
    else:
        jap_name=np.nan




    #type
    if soup.find(class_='type'):
        types=soup.find('span',class_='type').text
        types = re.sub(r'[\n\(\)]', '     ', types)
        type_=types[:5]

        df.iloc[i,3]=type_
        if type_=='Movie':
            df.iloc[i,4]=types[15:29]
        else:
            df.iloc[i,4]=types[11:15]
    else:
        df.iloc[i,3]=np.nan
    
    
    
    #Episodes
    
    #studio   
    studio=[]

    for ele in soup.find_all(class_='pure-1 md-1-5'):
        #if ele.find('a'):
        a=ele.find('a')
        if a!=None:
            studio.append(a.text)

            df.iloc[i,5]=studio[0]
    print(studio)
    
    
    #release season 
    if len(studio)==1:
        df.iloc[i,6]=np.nan
    if len(studio)==2:
        if studio[1][:4]=='Fall':
            df.iloc[i,6]=studio[1][:5]
        else:
            df.iloc[i,6]=studio[1][:6]
    #tags
    tags=[]
    for tag in soup.find_all(class_='pure-1 md-3-5'):
        if tag.find_all('ul'):
            for ul in tag.find_all('ul'):
                for li in ul.find_all('li'):
                    tags.append(li.text.strip())

            tags = ', '.join(tags)
            df.iloc[i,7]=tags
        else:
            df.iloc[i,7]=np.nan

    #rating
    if soup.find_all(class_='avgRating'):
        rating=soup.find(class_='avgRating').text.strip()
        rating=rating[:4]
        df.iloc[i,8]=rating
    else:
        df.iloc[i,8]=np.nan

    #release end year
    year=[]
    if soup.find(class_='iconYear'):
        for time in soup.find(class_='iconYear'):
            year.append(time.strip())

        release_year=year[0][:5]

        end_year=year[0][7:]

        df.iloc[i,10]=release_year
        df.iloc[i,11]=end_year
    else:
        df.iloc[i,10]=np.nan
        df.iloc[i,11]=np.nan

    #synopsis
    description=[]
    for desc in soup.find_all(class_='pure-1 md-3-5'):
        if desc.find('p'):
            description.append(desc.find('p').text.strip())
            df.iloc[i,12]=description[0]
        else:
            df.iloc[i,12]=np.nan

    #content warning
    warn=[]
    if soup.find_all(class_='tags tags--plain'):
        for tag in soup.find_all(class_='tags tags--plain'):
            for ul in tag.find_all('ul'):
                for li in ul.find_all('li'):
                    warn.append(li.text.strip())
        warn = ', '.join(warn)
    else:
        warn=np.nan


    df.iloc[i,13]=warn
    #releated manga
    manga=[]
    if soup.find_all(id='tabs--relations--manga--same_franchise'):
        for man in soup.find_all(id='tabs--relations--manga--same_franchise'):
            for mang in man.find_all(class_='RelatedEntry__name rounded-card__title'):
                manga.append(mang.text.strip())
        manga = ', '.join(manga)
        df.iloc[i,14]=manga
    else:
        df.iloc[i,14]=np.nan



    #releated anime
    anime=[]
    if soup.find_all(id='tabs--relations--anime--same_franchise'):
        for an in soup.find_all(id='tabs--relations--anime--same_franchise'):
            for mang in an.find_all(class_='RelatedEntry__name rounded-card__title'):
                anime.append(mang.text.strip())

        anime = ', '.join(anime)
        df.iloc[i,15]=anime
    else:
        df.iloc[i,15]=np.nan

    #voic actors
    voice=[]
    if soup.find_all(class_='CharacterCard__main'):
        for actor in soup.find_all(class_='CharacterCard__main'):
            for y in actor.find_all(class_='CharacterCard__content'):
                voice.append(y.text.title().strip())

        voice = ', '.join(voice)
        voice = re.sub(r'(\n \n)', ' : ', voice)
        df.iloc[i,16]=voice
    else:
        df.iloc[i,16]=np.nan

    #staff
    staff=[]
    if soup.find_all(class_='EntryPage__content__section EntryPage__content__section__staff castaff'):
        for desc in soup.find_all(class_='EntryPage__content__section EntryPage__content__section__staff castaff'):
            for y in desc.find_all(class_='CharacterCard__content'):
                staff.append(y.text.title().strip())

        staff = ', '.join(staff)
        staff = re.sub(r'[\n]', ' : ', staff)
        df.iloc[i,17]=staff
    else:
        df.iloc[i,17]=np.nan

    #rank
    ranks=[]
    for rank in soup.find_all(class_='pure-1 md-1-5')[4]:
        if rank!=None:
            ranks.append(rank)
            df.iloc[i,9]=ranks[0][7:-1]
        else:
            df.iloc[i,9]=np.nan
    print(ranks[0])

    #image
    img=[]
    for x in soup.find_all('img',class_="screenshots"):
         img.append(x['src'])
    df.iloc[i,18]=img[0]

    i=i+1


https://www.anime-planet.com/anime/spy-x-family
['WIT Studio', 'Spring 2022']

Rank #1

https://www.anime-planet.com/anime/demon-slayer-kimetsu-no-yaiba-entertainment-district-arc
['ufotable', 'Fall 2021']

Rank #2

https://www.anime-planet.com/anime/fruits-basket-the-final-season
['TMS Entertainment', 'Spring 2021']

Rank #3

https://www.anime-planet.com/anime/fullmetal-alchemist-brotherhood
['Bones', 'Spring 2009']

Rank #4

https://www.anime-planet.com/anime/mo-dao-zu-shi-3
['B.C MAY PICTURES']

Rank #5

https://www.anime-planet.com/anime/attack-on-titan-3rd-season-part-ii
['WIT Studio', 'Spring 2019']

Rank #6

https://www.anime-planet.com/anime/jujutsu-kaisen
['MAPPA', 'Fall 2020']

Rank #7

https://www.anime-planet.com/anime/haikyuu-karasuno-high-school-vs-shiratorizawa-academy
['Production I.G', 'Fall 2016']

Rank #8

https://www.anime-planet.com/anime/demon-slayer-kimetsu-no-yaiba-movie-mugen-train
['ufotable']

Rank #9

https://www.anime-planet.com/anime/attack-on-titan-the-fi


Rank #84

https://www.anime-planet.com/anime/jojos-bizarre-adventure-stone-ocean
['David Production', 'Fall 2021']

Rank #85

https://www.anime-planet.com/anime/kurokos-basketball-3
['Production I.G', 'Winter 2015']

Rank #87

https://www.anime-planet.com/anime/the-disastrous-life-of-saiki-k-2nd-season
['J.C.Staff', 'Winter 2018']

Rank #86

https://www.anime-planet.com/anime/horimiya
['CloverWorks', 'Winter 2021']

Rank #89

https://www.anime-planet.com/anime/hotarubi-no-mori-e
["Brain's Base"]

Rank #88

https://www.anime-planet.com/anime/attack-on-titan-no-regrets
['WIT Studio']

Rank #90

https://www.anime-planet.com/anime/dr-stone
['8 Pan', 'Summer 2019']

Rank #91

https://www.anime-planet.com/anime/monogatari-series-second-season
['SHAFT', 'Summer 2013']

Rank #92

https://www.anime-planet.com/anime/my-dress-up-darling
['CloverWorks', 'Winter 2022']

Rank #93

https://www.anime-planet.com/anime/hajime-no-ippo-rising
['MADHOUSE', 'Fall 2013']

Rank #95

https://www.anime-planet.


Rank #170

https://www.anime-planet.com/anime/miss-kobayashis-dragon-maid-s
['Kyoto Animation', 'Summer 2021']

Rank #171

https://www.anime-planet.com/anime/the-garden-of-sinners-movie-7-not-nothing-heart-murder-speculation-part-b
['ufotable']

Rank #173

https://www.anime-planet.com/anime/ascendance-of-a-bookworm-part-ii
['Ajia-do', 'Spring 2020']

Rank #174

https://www.anime-planet.com/anime/food-wars-the-second-plate
['J.C.Staff', 'Summer 2016']

Rank #175

https://www.anime-planet.com/anime/tokyo-revengers
['LIDEN FILMS', 'Spring 2021']

Rank #179

https://www.anime-planet.com/anime/hellsing-ultimate
['Graphinica']

Rank #176

https://www.anime-planet.com/anime/weathering-with-you
['CoMix Wave Films']

Rank #177

https://www.anime-planet.com/anime/golden-kamuy-season-3
['Geno Studio', 'Fall 2020']

Rank #178

https://www.anime-planet.com/anime/bungo-stray-dogs-2nd-season
['Bones', 'Fall 2016']

Rank #180

https://www.anime-planet.com/anime/ace-of-the-diamond-act-ii
['MADHOUSE', 


Rank #257

https://www.anime-planet.com/anime/kingdom-2
['Pierrot', 'Summer 2013']

Rank #254

https://www.anime-planet.com/anime/samurai-champloo
['Manglobe', 'Spring 2004']

Rank #256

https://www.anime-planet.com/anime/toilet-bound-hanako-kun
['Lerche', 'Winter 2020']

Rank #258

https://www.anime-planet.com/anime/kids-on-the-slope
['MAPPA', 'Spring 2012']

Rank #259

https://www.anime-planet.com/anime/grave-of-the-fireflies
['Studio Ghibli']

Rank #262

https://www.anime-planet.com/anime/a-place-further-than-the-universe
['MADHOUSE', 'Winter 2018']

Rank #261

https://www.anime-planet.com/anime/gintama-the-semi-final
['BN Pictures', 'Winter 2021']

Rank #255

https://www.anime-planet.com/anime/zoku-owarimonogatari
['SHAFT']

Rank #260

https://www.anime-planet.com/anime/major-season-2
['Studio Hibari', 'Fall 2005']

Rank #263

https://www.anime-planet.com/anime/overlord
['MADHOUSE', 'Summer 2015']

Rank #264

https://www.anime-planet.com/anime/detective-conan-movie-20-the-darkest-

https://www.anime-planet.com/anime/overlord-ii
['MADHOUSE', 'Winter 2018']

Rank #341

https://www.anime-planet.com/anime/summer-time-rendering
['OLM', 'Spring 2022']

Rank #312

https://www.anime-planet.com/anime/chuan-shu-zijiu-zhinan
[]

Rank #345

https://www.anime-planet.com/anime/kamisama-kiss-ova
['TMS Entertainment']

Rank #342

https://www.anime-planet.com/anime/ascendance-of-a-bookworm
['Ajia-do', 'Fall 2019']

Rank #343

https://www.anime-planet.com/anime/initial-d-fourth-stage
['A.C.G.T', 'Spring 2004']

Rank #344

https://www.anime-planet.com/anime/in-this-corner-of-the-world
['MAPPA']

Rank #347

https://www.anime-planet.com/anime/the-seven-deadly-sins-revival-of-the-commandments
['A-1 Pictures', 'Winter 2018']

Rank #348

https://www.anime-planet.com/anime/vivy-fluorite-eyes-song
['WIT Studio', 'Spring 2021']

Rank #353

https://www.anime-planet.com/anime/the-girl-who-leapt-through-time
['MADHOUSE']

Rank #350

https://www.anime-planet.com/anime/hunter-x-hunter-ova
['Nip

...


Rank #1,269

https://www.anime-planet.com/anime/bang-dream-movie-episode-of-roselia-part-ii-song-i-am
['SANZIGEN']

Rank #1,270

https://www.anime-planet.com/anime/the-kawai-complex-guide-to-manors-and-hostel-behavior-first-time
["Brain's Base"]

Rank #1,272

https://www.anime-planet.com/anime/aria-the-avvenire
['TYO Animations']

Rank #1,273

https://www.anime-planet.com/anime/maria-watches-over-us-season-4
['Studio DEEN', 'Winter 2009']

Rank #1,274

https://www.anime-planet.com/anime/cardcaptor-sakura
['MADHOUSE', 'Spring 1998']

Rank #1,275

https://www.anime-planet.com/anime/the-prince-of-tennis-ii-specials
['M.S.C.']

Rank #1,283

https://www.anime-planet.com/anime/strawberry-marshmallow-ova
['Daume']

Rank #1,276

https://www.anime-planet.com/anime/kingsglaive-final-fantasy-xv
['Visual Works']

Rank #1,277

https://www.anime-planet.com/anime/haibane-renmei
['Radix Ace Entertainment', 'Fall 2002']

Rank #1,278

https://www.anime-planet.com/anime/my-hero-academia-training-of-the-


Rank #1,356

https://www.anime-planet.com/anime/jueshi-wu-hun
['Soyep']

Rank #1,349

https://www.anime-planet.com/anime/yakitate-japan
['Sunrise', 'Fall 2004']

Rank #1,357

https://www.anime-planet.com/anime/hetalia-world-series-extra-episodes
['Studio DEEN']

Rank #1,358

https://www.anime-planet.com/anime/the-prince-of-tennis-another-story-kako-to-mirai-no-message
['M.S.C.']

Rank #1,359

https://www.anime-planet.com/anime/professor-layton-and-the-eternal-diva
['OLM']

Rank #1,361

https://www.anime-planet.com/anime/major-2nd
['OLM', 'Spring 2018']

Rank #1,362

https://www.anime-planet.com/anime/ling-long-incarnation-part-ii
['YHKT Entertainment']

Rank #1,364

https://www.anime-planet.com/anime/white-snake-2-green-snake
['Light Chaser Animation']

Rank #1,365

https://www.anime-planet.com/anime/kinos-journey-byouki-no-kuni-for-you-
['SHAFT']

Rank #1,366

https://www.anime-planet.com/anime/the-legend-of-the-legendary-heroes
['Zexcs', 'Summer 2010']

Rank #1,367

https://www.anim


Rank #1,442

https://www.anime-planet.com/anime/our-love-has-always-been-10-centimeters-apart
['Lay-duce', 'Fall 2017']

Rank #1,450

https://www.anime-planet.com/anime/arakawa-under-the-bridge-x-bridge
['SHAFT', 'Fall 2010']

Rank #1,443

https://www.anime-planet.com/anime/the-prince-of-tennis
['Production I.G', 'Fall 2001']

Rank #1,441

https://www.anime-planet.com/anime/asobi-asobase-ova
['Lerche']

Rank #1,445

https://www.anime-planet.com/anime/new-game
['Doga Kobo', 'Summer 2016']

Rank #1,444

https://www.anime-planet.com/anime/shounen-maid
['8-Bit', 'Spring 2016']

Rank #1,446

https://www.anime-planet.com/anime/knights-and-magic
['8-Bit', 'Summer 2017']

Rank #1,447

https://www.anime-planet.com/anime/eve-last-dance
[]

Rank #1,440

https://www.anime-planet.com/anime/ushio-and-tora-tv
['MAPPA', 'Summer 2015']

Rank #1,449

https://www.anime-planet.com/anime/digimon-adventure-tri-movie-1-reunion
['Toei Animation']

Rank #1,451

https://www.anime-planet.com/anime/linkin-park-b


Rank #1,540

https://www.anime-planet.com/anime/fate-stay-night
['Studio DEEN', 'Winter 2006']

Rank #1,525

https://www.anime-planet.com/anime/inu-x-boku-secret-service
['David Production', 'Winter 2012']

Rank #1,524

https://www.anime-planet.com/anime/space-dandy
['Bones', 'Winter 2014']

Rank #1,527

https://www.anime-planet.com/anime/ling-long-incarnation-zhong-zhang
['YHKT Entertainment']

Rank #1,528

https://www.anime-planet.com/anime/uchuu-senkan-yamato-2205-aratanaru-tabidachi-zenshou-take-off
['Staple Entertainment']

Rank #1,530

https://www.anime-planet.com/anime/the-twelve-kingdoms
['Pierrot', 'Spring 2002']

Rank #1,532

https://www.anime-planet.com/anime/jian-yu-chuanqi
['Ruo Hong Culture']

Rank #1,531

https://www.anime-planet.com/anime/kabaneri-of-the-iron-fortress
['WIT Studio', 'Spring 2016']

Rank #1,534

https://www.anime-planet.com/anime/one-piece-movie-6-baron-omatsuri-and-the-secret-island
['Toei Animation']

Rank #1,535

https://www.anime-planet.com/anime/bi

KeyboardInterrupt: 

In [ ]:
df.head(15)

,url,Name,Japanese_name,Type,Episodes,Studio,Release_season,Tags,Rating,Rank,Release_year,End_year,Description,Content_Warning,Related_Mange,Related_anime,Voice_actors,staff,Img
0,/anime/spy-x-family,SPY x FAMILY,NaN,TV,7+,WIT Studio,Spring,"Action, Comedy, Shounen, Assassins, Espionage,...",4.65,1,2022,?,Master spy Twilight is the best at what he doe...,NaN,"SPY x FAMILY, SPY x FAMILY: Kazoku no Shouzou ...",SPY x FAMILY: Part II,"Anya Forger : Atsumi Tanezaki, Loid Forger : T...","Tatsuya Endo : Original Creator, Kazuhiro Furu...",https://cdn.anime-planet.com/anime/primary/spy...
1,/anime/demon-slayer-kimetsu-no-yaiba-entertain...,Demon Slayer: Kimetsu no Yaiba - Entertainment...,Kimetsu no Yaiba: Yuukaku-hen,TV,11,ufotable,Fall,"Action, Adventure, Fantasy, Shounen, Demons, H...",4.61,2,2021,2022,Tanjiro and his friends accompany the Hashira ...,Explicit Violence,Demon Slayer: Kimetsu no Yaiba,Demon Slayer: Kimetsu no Yaiba - Swordsmith Vi...,"Inosuke Hashibira : Yoshitsugu Matsuoka, Nezuk...","Koyoharu Gotouge : Original Creator, Haruo Sot...",https://cdn.anime-planet.com/anime/primary/dem...
2,/anime/fruits-basket-the-final-season,Fruits Basket the Final Season,Fruits Basket the Final,TV,13,TMS Entertainment,Spring,"Drama, Fantasy, Romance, Shoujo, Animal Transf...",4.60,3,2021,,The final arc of Fruits Basket.,"Emotional Abuse,, Mature Themes,, Physical Abu...","Fruits Basket, Fruits Basket Another","Fruits Basket 1st Season, Fruits Basket 2nd Se...","Akito Sohma : Maaya Sakamoto, Kyo Sohma : Yuum...","Natsuki Takaya : Original Creator, Yoshihide I...",https://cdn.anime-planet.com/anime/primary/fru...
3,/anime/fullmetal-alchemist-brotherhood,Fullmetal Alchemist: Brotherhood,Hagane no Renkinjutsushi: Full Metal Alchemist,TV,64,Bones,Spring,"Action, Adventure, Drama, Fantasy, Mystery, Sh...",4.57,4,2009,2010,The foundation of alchemy is based on the law ...,"Animal Abuse,, Mature Themes,, Violence,, Dome...","Fullmetal Alchemist, Fullmetal Alchemist (Ligh...","Fullmetal Alchemist: Brotherhood Specials, Ful...","Alphonse Elric : Rie Kugimiya, Edward Elric : ...","Hiromu Arakawa : Original Creator, Yasuhiro Ir...",https://cdn.anime-planet.com/anime/primary/ful...
4,/anime/mo-dao-zu-shi-3,Mo Dao Zu Shi 3,The Founder of Diabolism 3,Web,12,B.C MAY PICTURES,NaN,"Fantasy, Ancient China, Chinese Animation, Cul...",4.56,5,2021,,The third season of Mo Dao Zu Shi.,NaN,Grandmaster of Demonic Cultivation: Mo Dao Zu ...,"Mo Dao Zu Shi 2, Mo Dao Zu Shi Q","Lan Wangji, Wei Wuxian, Jiang Cheng, Jin Guang...","Mo Xiang Tong Xiu : Original Creator, Xiong Ke...",https://cdn.anime-planet.com/anime/primary/mo-...
5,/anime/attack-on-titan-3rd-season-part-ii,Attack on Titan 3rd Season: Part II,Shingeki no Kyojin Season 3: Part II,TV,10,WIT Studio,Spring,"Action, Fantasy, Horror, Shounen, Dark Fantasy...",4.55,6,2019,,The battle to retake Wall Maria begins now! Wi...,"Cannibalism,, Explicit Violence","Attack on Titan, Attack on Titan: End of the W...","Attack on Titan, Attack on Titan 2nd Season, A...","Armin Arlelt : Marina Inoue, Eren Jaeger : Yuu...","Hajime Isayama : Original Creator, Tetsurou Ar...",https://cdn.anime-planet.com/anime/primary/att...
6,/anime/jujutsu-kaisen,Jujutsu Kaisen,NaN,TV,24,MAPPA,Fall,"Action, Horror, Shounen, Body Sharing, Curse, ...",4.54,7,2020,2021,Although Yuji Itadori looks like your average ...,Explicit Violence,"Jujutsu Kaisen 0, Jujutsu Kaisen","Jujutsu Kaisen (2018), Juju Sanpo, Eve: Kaikai...","Megumi Fushiguro : Yuuma Uchida, Nobara Kugisa...","Gege Akutami : Original Creator, Seong-Hu Park...",https://cdn.anime-planet.com/anime/primary/juj...
7,/anime/haikyuu-karasuno-high-school-vs-shirato...,Haikyuu!! Karasuno High School vs Shiratorizaw...,Haikyuu!! Karasuno Koukou vs Shiratorizawa Gak...,TV,10,Production I.G,Fall,"Shounen, Sports, Animeism, School Club, School...",4.53,8,2016,,"Picking up where the second season ended, the ...",NaN,"Haikyuu!! (Pilot), Haikyuu!!, Haikyuu-bu!!","Haikyuu!!, Haikyuu!! Lev Ken

In [ ]:
df[df['url']=='/anime/servant-x-service']

,url,Name,Japanese_name,Type,Episodes,Studio,Release_season,Tags,Rating,Rank,Release_year,End_year,Description,Content_Warning,Related_Mange,Related_anime,Voice_actors,staff,Img
1342,/anime/servant-x-service,Servant x Service,NaN,TV,13,A-1 Pictures,Summer,"Comedy, Romance, Seinen, Slice of Life, Cowork...",3.89,"1,342",2013,,"When Lucy was born, her parents couldn’t decid...",NaN,Servant x Service,NaN,"Lucy Yamagami : Ai Kayano, Megumi Chihaya : Ak...","Karino Takatsu : Original Creator, Yasutaka Ya...",https://cdn.anime-planet.com/anime/primary/ser...
